In [ ]:
import pandas as pd

ticker = input("どの銘柄を読み込みますか？")
df_q = pd.read_csv(f"df_q/{ticker}.csv")
df_day = pd.read_csv(f"df_day/{ticker}.csv")
df_c = pd.read_csv(f"df_c/{ticker}.csv")

In [ ]:
df_q

In [5]:
#欠損値処理のため、nanをインポート
from numpy import nan

#求めたい四半期から過去の四半期を作成し、成長率を求める関数
def slide(present):
    past = present.shift(-1)
    growth_rate  = (present - past)/past *100
    #四半期の一つ分ずらしているため、ずらして要らなくなったデータ(nan)を除去
    return growth_rate.dropna()

In [ ]:
#売上成長率
gp1_slide = slide(df_q["net_sales"])
gp1_slide 

In [7]:
#加重平均を求める関数
def w_ave(x):
    sum_wa = 0
    count=0
    
    #シリーズ型のためvalueのみ取り出している
    for quarter in x.values:
        count += 1
        #四半期ごとに重みをつける
        weight = quarter * count
        #重みをつけた上で年度ごとの数値を足し合わせる
        sum_wa += weight 
        
    #加重平均した値を算出
    return sum_wa / ((1/2)*count*(count+1))

In [ ]:
gp1_nsgr =w_ave(gp1_slide)
#売上成長率
gp1_nsgr

In [ ]:
#営業利益成長率
gp2_slide = slide(df_q["operating_income"])
gp2_slide

gp2_oigr =w_ave(gp2_slide)
#営業利益成長率
gp2_oigr

In [ ]:
#ROE
pr1_roe = w_ave(df_q["roe"])       
#ROE
pr1_roe 

In [ ]:
df_q["roe"]

In [ ]:
#売上高営業利益率
pr2_om = w_ave(df_q["operating_margin"])       
#売上高営業利益率
pr2_om

In [ ]:
df_q["operating_margin"]

In [28]:
def w_ave(x):
    sum_wa = 0
    count=0
    
    #欠損値を削除
    x_dropna = x.dropna()
    
    #シリーズ型のためvalueのみ取り出している
    for quarter in x_dropna.values:
        count += 1
        weight = quarter * count
        sum_wa += weight
    
    #加えて、もしデータが全て欠損値だった場合nanと返す
    if sum_wa == 0 :
        return nan
    else:
        return sum_wa / ((1/2)*count*(count+1))

In [ ]:
pr2_om = w_ave(df_q["operating_margin"])       
#売上高営業利益率
pr2_om

In [ ]:
#1人当たり売上高
lp1_nspe = w_ave(df_q["net_sales_per_employee"])
#1人当たり売上高
lp1_nspe

In [ ]:
#1人当たり営業CF
lp2_oipe = w_ave(df_q["operating_income_per_employee"])
#1人当たり営業CF
lp2_oipe

In [ ]:
#総資産回転率
e1_tat = w_ave(df_q["total_asset_turnover"])
#総資産回転率
e1_tat

In [ ]:
#CCC(キャッシュコンバージョンサイクル) = 売上債権回転期間 + 棚卸資産回転期間 – 支払債務回転期間
e2_df = df_q[["accounts_receivable_turnover","inventory_turnover","trade_payable_turnover"]]
e2_ccc = w_ave(e2_df["trade_payable_turnover"])+w_ave(e2_df["inventory_turnover"])-w_ave(e2_df["accounts_receivable_turnover"])
#CCC(キャッシュコンバージョンサイクル) = 売上債権回転期間 + 棚卸資産回転期間 – 支払債務回転期間
e2_ccc

In [ ]:
#自己資本比率
s1_er = w_ave(df_q["equity_ratio"])
#自己資本比率
s1_er 

In [ ]:
#ネットD/Eレシオ = 純有利子負債 net_debt/ 自己資本 equity
s2_df = df_q[["net_debt","equity"]]
s2_nder = w_ave(s2_df["net_debt"]/s2_df["equity"])
#ネットD/Eレシオ = 純有利子負債 net_debt/ 自己資本 equity
s2_nder

In [ ]:
#配当性向 Dividend payout ratio (%）＝ 1株あたり配当金 Dividends per share ÷1株あたり純利益（EPS）×100
sr1_df = df_q[["dividend","eps_actual"]]
sr1_dqr = w_ave(sr1_df["dividend"] /sr1_df["eps_actual"] * 100)
#配当性向 Dividend payout ratio (%）＝ 1株あたり配当金 Dividends per share ÷1株あたり純利益（EPS）×100
sr1_dqr

In [ ]:
sr1_df

In [ ]:
#株主資本配当率 DOE
sr2_doe = w_ave(df_q["doe"])
#株主資本配当率 DOE
sr2_doe

In [ ]:
df_q["doe"]

In [ ]:
def dc_merge(daily,current):
    if current[0] is None:
        return daily
    else : 
        dcm = pd.concat([daily,current],axis = 0)
        return dcm


df_cg = df_day[["market_capital","num_of_shares"]]
cg_d = df_cg["market_capital"]/df_cg["num_of_shares"]
cg_c = df_c["stockprice"]
cg_merge = dc_merge(cg_d,cg_c)
cg_before = cg_merge
cg_after = cg_merge.shift(-1)
#slide関数では100倍するため注意
cg_ab = (cg_after - cg_before)/cg_before
cg = w_ave(cg_ab)
cg 